In [2]:
import warnings
warnings.filterwarnings('ignore')

import sys
from pytz import UTC
import logbook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from logbook import (NestedSetup, NullHandler, Logger, 
                     StreamHandler, StderrHandler, 
                     INFO, WARNING, DEBUG, ERROR)

from zipline import run_algorithm
from zipline.api import (attach_pipeline,
                         date_rules,
                         time_rules,
                         get_datetime,
                         order_target_percent,
                         pipeline_output,
                         record, schedule_function,
                         get_open_orders,
                         calendars,
                         set_commission, 
                         set_slippage)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, objective_functions
from pypfopt import expected_returns
from pypfopt.exceptions import OptimizationError

from pyfolio.utils import extract_rets_pos_txn_from_zipline

sns.set_style('whitegrid')

# setup stdout logging
format_string = '[{record.time: %H:%M:%S.%f}]: {record.level_name}: {record.message}'
zipline_logging = NestedSetup([NullHandler(level=DEBUG),
                               StreamHandler(sys.stdout, format_string=format_string, level=INFO),
                               StreamHandler(sys.stdout, format_string=format_string, level=WARNING),
                               StreamHandler(sys.stderr, level=ERROR)])
zipline_logging.push_application()
log = Logger('Algorithm')

# Settings
MONTH = 21
YEAR = 12 * MONTH
N_LONGS = 50
N_SHORTS = 50
MIN_POS = 5
VOL_SCREEN = 1000

#start = pd.Timestamp('2013-01-01', tz=UTC)
#end = pd.Timestamp('2017-01-01', tz=UTC)
start = pd.Timestamp('2013-01-01')
end = pd.Timestamp('2017-01-01')
capital_base = 1e7

In [3]:
class MeanReversion(CustomFactor):
    """Compute ratio of latest monthly return to 12m average,
       normalized by std dev of monthly returns"""
    inputs = [Returns(window_length=MONTH)]
    window_length = YEAR

    def compute(self, today, assets, out, monthly_returns):
        df = pd.DataFrame(monthly_returns)
        factor = df.iloc[-1].sub(df.mean()).div(df.std())
        out[:] = factor
        
def compute_factors():
    """Create factor pipeline incl. mean reversion,
        filtered by 30d Dollar Volume; capture factor ranks"""
    mean_reversion = MeanReversion()
    dollar_volume = AverageDollarVolume(window_length=30)
    return Pipeline(columns={'longs'  : mean_reversion.bottom(N_LONGS),
                             'shorts' : mean_reversion.top(N_SHORTS),
                             'ranking': mean_reversion.rank(ascending=False)},
                    screen=dollar_volume.top(VOL_SCREEN))

def before_trading_start(context, data):
    """Run factor pipeline"""
    context.factor_data = pipeline_output('factor_pipeline')
    record(factor_data=context.factor_data.ranking)
    assets = context.factor_data.index
    record(prices=data.current(assets, 'price'))
    
def exec_trades(data, positions):
    """Place orders for assets using target portfolio percentage"""
    for asset, target_percent in positions.items():
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)
            
def optimize_weights(prices, short=False):

    returns = expected_returns.mean_historical_return(prices=prices, frequency=252) #统计历史年化回报
    cov = risk_models.sample_cov(prices=prices, frequency=252)                      #计算【股票回报率】的【年化样本协方差】？？

    # get weights that maximize the Sharpe ratio
    ef = EfficientFrontier(expected_returns=returns, cov_matrix=cov, weight_bounds=(0, 1), solver='SCS')
    ef.max_sharpe()
    if short:
        return {asset: -weight for asset, weight in ef.clean_weights().items()}
    else:
        return ef.clean_weights()
    
def initialize(context):
    """Setup: register pipeline, schedule rebalancing,
        and set trading params"""
    attach_pipeline(compute_factors(), 'factor_pipeline')
    schedule_function(rebalance, date_rules.week_start(), time_rules.market_open(), calendar=calendars.US_EQUITIES)

    set_commission(us_equities=commission.PerShare(cost=0.00075, min_trade_cost=.01))
    set_slippage(us_equities=slippage.VolumeShareSlippage(volume_limit=0.0025, price_impact=0.01))



In [11]:
start = pd.Timestamp('2013-01-01')
end = pd.Timestamp('2013-02-10')

def rebalance(context, data):
    """Compute long, short and obsolete holdings; place orders"""

    factor_data = context.factor_data
    assets = factor_data.index

    longs = assets[factor_data.longs]
    shorts = assets[factor_data.shorts]
    
    divest = context.portfolio.positions.keys() - longs.union(shorts)
    exec_trades(data, positions={asset: 0 for asset in divest})
    log.info('{} | {:11,.0f}'.format(get_datetime().date(), context.portfolio.portfolio_value))

    # get price history
    prices = data.history(assets, fields='price',
                          bar_count=252+1, # for 1 year of returns 
                          frequency='1d')
    
    # get optimal weights if sufficient candidates
    if len(longs) > MIN_POS and len(shorts) > MIN_POS:
        try:
            long_weights = optimize_weights(prices.loc[:, longs])
            short_weights = optimize_weights(prices.loc[:, shorts], short=True)
            log.info(long_weights.values())
            log.info(short_weights.values())
            
            exec_trades(data, positions=long_weights)
            exec_trades(data, positions=short_weights)
        except Exception as e:
            log.warn('{} {}'.format(get_datetime().date(), e))
    
    # exit remaining positions, 卖掉上一周期的持仓
    divest_pf = {asset: 0 for asset in context.portfolio.positions.keys()}
    exec_trades(data, positions=divest_pf)
    
backtest = run_algorithm(start=start, end=end,
                         initialize=initialize, before_trading_start=before_trading_start,
                         bundle='quandl', capital_base=capital_base)

[ 03:34:40.631167]: INFO: 2013-01-07 |  10,000,000
[ 03:34:42.710424]: INFO: odict_values([0.1661, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24376, 0.0, 0.0, 0.0, 0.0, 0.44275, 0.0, 0.0, 0.0, 0.0, 0.10069, 0.0, 0.0467, 0.0])
[ 03:34:42.711074]: INFO: dict_values([-0.14825, -0.0, -0.13386, -0.0, -0.66881, -0.0, -0.04907])
[ 03:34:45.631605]: INFO: 2013-01-14 |   9,922,439
[ 03:34:47.108529]: INFO: odict_values([0.14651, 0.14971, 0.22553, 0.0, 0.0, 0.0, 0.10139, 0.0, 0.0, 0.11717, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2597, 0.0])
[ 03:34:47.109180]: INFO: dict_values([-0.05949, -0.67216, -0.0, -0.11493, -0.0, -0.15343, -0.0, -0.0, -0.0, -0.0])
[ 03:34:47.922659]: INFO: 2013-01-22 |   9,913,618
[ 03:34:49.402855]: INFO: odict_values([0.0, 0.46172, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53831, 0.0, 0.0, 0.0, 0.0])
[ 03:34:49.403489]: INFO: dict_values([-0.0, -0.09369, -0.0, -0.33587, -0.0, -0.2734, -0.0, -0.0, -0.10964, -0.1874])
[ 03:34:50.083569]: INFO: 2013-01-28 |   9,968,673
[ 03:34:51.579034

In [ ]:
[ 03:31:29.230721]: INFO: 2013-01-07 |  10,000,000
[ 03:31:31.308118]: INFO: odict_values([0.1661, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24376, 0.0, 0.0, 0.0, 0.0, 0.44275, 0.0, 0.0, 0.0, 0.0, 0.10069, 0.0, 0.0467, 0.0])
[ 03:31:31.308738]: INFO: dict_values([-0.14825, -0.0, -0.13386, -0.0, -0.66881, -0.0, -0.04907])